In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os
import numpy as np
import parser
import math
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
import generar_jobs
from sklearn.metrics import accuracy_score

In [2]:
# Parámetros hardcodeados del problema a estudiar
taskAmount = 128
machineAmount = 4
taskHeterogeneity = 0
machineHeterogeneity = 0
consistencyType = 0
accuracyScores = []
classifiers = []

dimension = taskAmount * machineAmount

# Referencia: https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
ns = 600 # Cantidad de instancias de entrenamiento
ni = dimension
no = 1 # Cantidad de salidas
alpha = 2
hiddenLayerAmount = int(math.ceil(ns / (alpha * (ni + no))))
# Cada capa oculta tiene una cantidad de neuronas intermedia entre la cantidad de salidas y entradas
# Se pasa a una tupla para mandarlo al constructor del MLPClassifier
hiddenLayerNeuronAmount = tuple([int(math.ceil((taskAmount - no) / 2))] * hiddenLayerAmount)


In [5]:
# Path base para todos los clasificadores
model_base_path ='./models/' + str(taskAmount) + 'x' + str(machineAmount) \
    + '-' + str(taskHeterogeneity) + str(machineHeterogeneity) + str(consistencyType) \
    + '/'
baseDir = './ejemplos-entrenamiento-unificados/128x4-000/'
model_file_prefix = 'classifier'
model_file_extension = '.pkl'

for i in range(0, taskAmount):
    # Se crea clasificador o se carga si ya existe
    try:
        classifier = joblib.load(model_base_path + model_file_prefix + str(i) + model_file_extension)
    except Exception:
        print('El clasificador para la salida ' + str(i) + ' no existía.')
        classifier = MLPClassifier(solver='lbfgs', alpha=1e-5,
                            hidden_layer_sizes=hiddenLayerNeuronAmount, random_state=1)    
    finally:
        # Se agrega a lista de clasificadores
        classifiers.append(classifier)

for i in range(0, taskAmount):
    # Se cargan los datos
    TRAINING_FILE = baseDir + 'training/' + str(i) + '.csv'
    TEST_FILE = baseDir + 'test/' + str(i) + '.csv'
    training_set = pd.read_csv(TEST_FILE, header=None, delimiter=',')
    test_set = pd.read_csv(TRAINING_FILE, header=None, delimiter=',')    
    # Se crea dataframe para datos y se separa el target
    df_training = pd.DataFrame(training_set)
    y_training = df_training.iloc[:,-1]
    # Se cargan datos de validacion
    df_test = pd.DataFrame(test_set)
    y_test = df_test.iloc[:,-1]
    # Se entrena clasificador con datos
    classifiers[i].fit(df_training.iloc[:,:-1], y_training)
    # Se genera el directorio del clasificador si no existe
    generar_jobs.generateDir(model_base_path)
    # Se persiste
    joblib.dump(classifiers[i], model_base_path + model_file_prefix + str(i) + model_file_extension)
    # Determino accuracy para este clasificador
    results = []
    for j in range(0,len(df_test)):
        # Clasifico cada instancia del archivo de test, y agrego la clasificacion
        # a un array de resultados
        results.append(classifiers[i].predict(df_test.iloc[j][:-1].values.reshape(1,-1)))
    # Comparo los resultados con los esperados    
    accuracy = accuracy_score(y_test, results)
    print("Precision de clasificador: " + str(accuracy))        
    # Agrego accuracy a lista de accuracies
    accuracyScores.append(accuracy)
                                                                             
# Calculo promedio de accuracy para todos los clasificadores
promedio = 0.
scoreAmount = len(accuracyScores)
for i in range(0,scoreAmount):
    promedio += accuracyScores[i]
promedio /= scoreAmount
print ('El promedio de accuracy es de {}'.format(promedio))

El clasificador para la salida 0 no existía.
El clasificador para la salida 1 no existía.
El clasificador para la salida 2 no existía.
El clasificador para la salida 3 no existía.
El clasificador para la salida 4 no existía.
El clasificador para la salida 5 no existía.
El clasificador para la salida 6 no existía.
El clasificador para la salida 7 no existía.
El clasificador para la salida 8 no existía.
El clasificador para la salida 9 no existía.
El clasificador para la salida 10 no existía.
El clasificador para la salida 11 no existía.
El clasificador para la salida 12 no existía.
El clasificador para la salida 13 no existía.
El clasificador para la salida 14 no existía.
El clasificador para la salida 15 no existía.
El clasificador para la salida 16 no existía.
El clasificador para la salida 17 no existía.
El clasificador para la salida 18 no existía.
El clasificador para la salida 19 no existía.
El clasificador para la salida 20 no existía.
El clasificador para la salida 21 no existía